# The PolInSAR Course - June 3rd, 2024
# SAR Polarimetry (PolSAR) 
# Part 2: Eigenvalues of the Polarimetric Coherency Matrix and the Entropy/Anisotropy/Alpha decomposition

* Acquisition: Nkok (Gabon), DLR's F-SAR, L-band

* Path to images: /projects/data/polsar/

* SLC (single-look complex) images:
    * HH: slc_16afrisr0107_Lhh_tcal_test.rat
    * HV: slc_16afrisr0107_Lhv_tcal_test.rat
    * VH: slc_16afrisr0107_Lvh_tcal_test.rat
    * VV: slc_16afrisr0107_Lvv_tcal_test.rat

Tips:
- use a function that performs the multilook (correlation) operation on a moving window with (looksa x looksr) pixels in range - azimuth
- focus on a azimuth - range block within pixels [5000, 15000] and [0, 2000], respectively.

In [1]:
# --- Download exercise data & import reader function
from pysarpro import io, data
from pysarpro.io import rrat

data.download_all(directory="/projects", pattern=r'^data/polsar')

# --- Import useful libaries, functions, and modules
import sys
sys.path.append('/projects/src/')
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import uniform_filter
%matplotlib widget

**Auxiliary functions**

`HSV_colormap_to_rgb`: Generates and HSV composite representation based on a given colormap.

In [ ]:
def HSV_colormap_to_rgb(colormap, h, s, v):
    """
    Makes an HSV-like RGB representation based on the given colormap instead
    of 'hsv' colormap.
    
    See https://en.wikipedia.org/wiki/HSL_and_HSV

    Parameters
    ----------
    colormap : function
        Colormap function. Takes the values in 'h' array and returns an RGBA
        value for each point. The ones in matplotlib.cm should be compatible
    h : ndarray
        Hue values. Usually between 0 and 1.0.
    s : ndarray
        Saturation values. Between 0 and 1.0.
    v : ndarray
        Value values. Between 0 and 1.0.

    Returns
    -------
    rgb: ndarray
        An array with the same shape as input + (3,) representing the RGB.
    """
    # Generate color between given colormap (colormap(h)) and white (ones)
    # according to the given saturation
    tmp = (1-s)[..., np.newaxis]*np.ones(3) + s[..., np.newaxis] * colormap(h)[...,:3]
    # Scale it by value
    return v[..., np.newaxis] * tmp

`calculate_covariance`: Calculates the covariance between two images while performing a multi-looking operation.

In [2]:
def calculate_covariance(im1, im2, looksa, looksr):
    
     # ... apply definition
    corr = uniform_filter( np.real(im1*np.conj(im2)), [looksa, looksr] ) + \
        1j*uniform_filter( np.imag(im1*np.conj(im2)), [looksa, looksr] )
    
    # ... and back to main
    return corr

`calculate_eigenvalues_3`: Computes the eigenvalues of a 3x3 matrix analytically. 

In [ ]:
def calculate_eigenvalues_3(T11, T12, T13, T22, T23, T33):

    # Calculate and order (from max to min) the eigenvalues of a 3x3 hermitian matrix in closed-form.
    # Inputs can be 2D az - rg (rows - columns).

    # get dimensions
    dims = T11.shape

    # calculate auxiliary quantities
    A = T11*T22 + T11*T33 + T22*T33 - T12*np.conj(T12) - T13*np.conj(T13) - T23*np.conj(T23)
    B = T11**2 - T11*T22 + T22**2 -T11*T33 -T22*T33 + T33**2 + 3*T12*np.conj(T12) + 3*T13*np.conj(T13) + 3*T23*np.conj(T23)

    DET = T11*T22*T33 - T33*T12*np.conj(T12) - T22*T13*np.conj(T13) - T11*T23*np.conj(T23) + T12*np.conj(T13)*T23 + np.conj(T12)*T13*np.conj(T23)  
    TR = T11 + T22 + T33 
    Z = 27*DET-9*A*TR + 2*TR**3 + np.sqrt((27*DET-9*A*TR + 2*TR**3)**2-4*B**3)
    
    del DET
    
    # ... and here they are:
    LA = ( 1/3.*TR + 2**(1/3.)*B/(3*Z**(1/3.)) + Z**(1/3.)/(3*2**(1/3.)) )
    LB = ( 1/3.*TR - (1+1j*np.sqrt(3))*B/(3*2**(2/3.)*Z**(1/3.)) - (1-1j*np.sqrt(3))*Z**(1/3.)/(6*2**(1/3.)) )
    LC = ( 1/3.*TR - (1-1j*np.sqrt(3))*B/(3*2**(2/3.)*Z**(1/3.)) - (1+1j*np.sqrt(3))*Z**(1/3.)/(6*2**(1/3.)) )
    
    # now order them:
    dumm = np.zeros((dims[0], dims[1], 3), 'float32')
    dumm [:, :, 0] = np.real(LA)
    dumm [:, :, 1] = np.real(LB)
    dumm [:, :, 2] = np.real(LC)
    
    del LA, LB, LC  
    
    L1 = np.max(dumm, axis = 2)
    L3 = np.min(dumm, axis = 2)
    L2 = np.sum(dumm, axis = 2) - L1 - L3
    
    del dumm
    
    return L1, L2, L3
    

`calculate_eigenvectors_3`: Computes the eigenvectors of a 3x3 matrix analytically. 

In [ ]:
def calculate_eigenvectors_3(T11, T12, T13, T22, T23, T33, L1, L2, L3) :

    # Calculate the eigenvectors corresponding to the eigenvalues (L1, L2, L3)
    # of a 3x3 matrix 
    # Inputs can be 2D az - rg (rows - columns).

    # get dimensions
    dims = T11.shape    
    
    # first eigenvector - corresponds to the maximum eigenvalue L1
    U1 = np.ones((dims[0], dims[1], 3), 'complex64')
    U1[:, :, 0] = (L1 -T33)/np.conj(T13) + (((L1-T33)*np.conj(T12) + np.conj(T13)*T23)*np.conj(T23))/ \
                    (((T22-L1)*np.conj(T13) - np.conj(T12)*np.conj(T23))*np.conj(T13))
    U1[:, :, 1] = -((L1-T33)*np.conj(T12)+np.conj(T13)*T23) / ((T22-L1)*np.conj(T13) - np.conj(T12)*np.conj(T23))
    
    # second eigenvector - corresponds to the eigenvalue L2
    U2 = np.ones((dims[0], dims[1], 3), 'complex64')
    U2[:, :, 0] = (L2 -T33)/np.conj(T13) + (((L2-T33)*np.conj(T12) + np.conj(T13)*T23)*np.conj(T23))/ \
                    (((T22-L2)*np.conj(T13) - np.conj(T12)*np.conj(T23))*np.conj(T13))
    U2[:, :, 1] = -((L2-T33)*np.conj(T12)+np.conj(T13)*T23) / ((T22-L2)*np.conj(T13) - np.conj(T12)*np.conj(T23))
    
    # third eigenvector - corresponds to the minimum eigenvalue L3
    U3 = np.ones((dims[0], dims[1], 3), 'complex64')
    U3[:, :, 0] = (L3 -T33)/np.conj(T13) + (((L3-T33)*np.conj(T12) + np.conj(T13)*T23)*np.conj(T23))/ \
                    (((T22-L3)*np.conj(T13) - np.conj(T12)*np.conj(T23))*np.conj(T13))
    U3[:, :, 1] = -((L3-T33)*np.conj(T12)+np.conj(T13)*T23) / ((T22-L3)*np.conj(T13) - np.conj(T12)*np.conj(T23))   
    
    # normalize to get orthonormal eigenvectors
    norm1 = np.sqrt( np.abs(U1[:,:,0])**2 + np.abs(U1[:,:,1])**2 + np.abs(U1[:,:,2])**2)
    norm2 = np.sqrt( np.abs(U2[:,:,0])**2 + np.abs(U2[:,:,1])**2 + np.abs(U2[:,:,2])**2)    
    norm3 = np.sqrt( np.abs(U3[:,:,0])**2 + np.abs(U3[:,:,1])**2 + np.abs(U3[:,:,2])**2)        
    for nn in range(3):
        U1[:,:,nn] = U1[:,:,nn] / norm1
        U2[:,:,nn] = U2[:,:,nn] / norm2
        U3[:,:,nn] = U3[:,:,nn] / norm3
        
    del norm1, norm2, norm3     
    
    return U1, U2, U3


**Input parameters**

In [ ]:
# path to the data
...
# define the number of looks 
...
...

**Step 1: Load data**

In [ ]:
...
...
...

In [ ]:
# check shape
...

**Step 2: Calculate the necessary elements of the coherency matrix**

In [ ]:
# -- compute the Pauli components
pauli1 = ...
pauli2 = ...
pauli3 = ...

In [ ]:
# -- compute the elements of the coherency matrix
T11 = ...
T22 = ...
T33 = ...
T12 = ...
T13 = ...
T23 = ...

In [ ]:
# -- delete unused variables
...
...

**Step 3: Calculate eigenvalues**

In [ ]:
...

In [ ]:
# check shape
...

**Step 4: Calculate entropy**

In [ ]:
# -- compute the probabilities associated with each eigenvalue
pr1 = ...
pr2 = ...
pr3 = ...

In [ ]:
# -- compute the entropy
entropy = ...

**Step 5: Calculate anisotropy** 

In [ ]:
# -- compute the anisotropy (related to the minimum and intermediate eigenvalues)
# A = 0 when lambda2 = lambda3
# A = 1 when lambda2 >> lambda3 
anisotropy = ...

**Step 6: Calculate eigenvectors**

In [ ]:
# -- compute the eigenvectors
...

In [ ]:
# check shape
...

In [ ]:
# -- delete unused variables
...

**Step 7: Calculate mean alpha angle**

In [ ]:
# -- extract the alpha angles
alpha1 = ...  # [rad
alpha2 = ... 
alpha3 = ... 

In [ ]:
# -- delete unused variables
...

In [ ]:
# -- compute the mean alpha angle
...
# -- transfer to degrees
...

**Step 8: Plots!**

In [ ]:
# Calculations for Paulis RGB:
# -- define the 3D array for the Pauli representation
...
# -- fill the array, clipping the values between 0 and 2.5xmean(amplitude)
...
# -- normalisation: values between 0 and 1
...

In [ ]:
# Plot: Pauli RGB and eigenvalue probabilities
...

In [ ]:
# Plot: H, A, alpha
...

In [ ]:
# HSI Color Representation:


 HSI Color Representation:
- H (hue):  mean alpha angle
- S (saturation): 
     - Case 1: saturation = 1: always full colorscale
     - Case 2:  saturation = 1 - entropy
          - when entropy = 0: then saturation = 1: full colorscale
          - when entropy = 1: then saturation = 0: grayscale
- I (intensity): amplitude of total power

In [ ]:
# Hue: mean alpha angle
# normalize the mean alpha angle: it has to be between 0 and 1 --> divide by 90 degrees
...
# Import the colormap for plotting alpha
...

# Intensity: normalize the amplitude
...

# Saturation
# Case 1)
...
# Case 2)
...

In [ ]:
# Generate the HSV colormaps 

# Case 1
...
# Case 2
...


In [ ]:
# Plot: HSI representations
...